In [1]:
import pandas as pd

In [13]:
from datetime import datetime, timedelta

In [3]:
relevant_days = (datetime(2023, 6, 4) - datetime(2023, 1, 1)).days

In [4]:
relevant_days = (datetime(2023, 6, 4) - datetime(2023, 1, 1)).days

In [9]:
reorg_date = datetime(2023, 1, 19)

In [10]:
df = pd.read_csv(
    "src_data/u_bkop_incident_230603.csv",
    parse_dates=["created", "resolved"],
    low_memory=False
)

In [11]:
cdf = df[(df["category"] == "Cataloging")|(df["assignment_group"] == "BKOPS CAT")]

In [14]:
before_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["created"] < datetime(2023, 1, 19)) & (cdf["created"] >= reorg_date - timedelta(relevant_days)) & (cdf["resolved"].notnull())]

In [15]:
before_df.shape[0]

827

In [16]:
before_df["resolved"].unique()

<DatetimeArray>
['2022-08-18 12:05:05', '2022-08-18 12:48:18', '2022-08-19 07:35:52',
 '2022-08-18 15:10:34', '2022-08-18 16:19:13', '2022-08-19 10:30:15',
 '2022-08-19 10:27:41', '2022-08-19 14:28:02', '2022-08-22 14:15:29',
 '2022-08-22 13:39:02',
 ...
 '2023-01-19 09:47:53', '2023-01-23 09:17:10', '2023-01-20 16:08:51',
 '2023-01-17 15:59:04', '2023-01-26 14:12:56', '2023-01-18 15:24:40',
 '2023-01-23 10:28:14', '2023-01-31 08:54:31', '2023-01-19 09:46:00',
 '2023-01-27 12:18:09']
Length: 826, dtype: datetime64[ns]

In [17]:
after_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["created"] >= datetime(2023, 1, 19)) & (cdf["resolved"].notnull())]

In [18]:
after_df.shape[0]

1045

In [19]:
after_df["resolved"].unique()

<DatetimeArray>
['2023-01-19 16:52:31', '2023-01-19 13:34:33', '2023-01-23 15:35:38',
 '2023-01-23 15:41:01', '2023-01-20 11:18:22', '2023-01-20 11:04:47',
 '2023-01-20 11:17:03', '2023-01-20 10:32:29', '2023-01-20 10:18:42',
 '2023-01-23 09:00:14',
 ...
 '2023-06-01 10:10:36', '2023-06-01 09:25:47', '2023-06-01 11:52:41',
 '2023-06-02 12:18:13', '2023-06-01 15:22:57', '2023-06-02 09:56:36',
 '2023-06-02 09:58:57', '2023-06-02 10:32:44', '2023-06-02 14:52:23',
 '2023-06-02 12:54:16']
Length: 1045, dtype: datetime64[ns]

In [20]:
before_df["time2resolve"] = before_df["resolved"] - before_df["created"]

C:\Users\tomaszkalata\AppData\Local\Temp\ipykernel_5300\1901942124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_df["time2resolve"] = before_df["resolved"] - before_df["created"]


In [21]:
before_df["time2resolve"].describe()

count                           827
mean     33 days 06:57:33.574365175
std      50 days 13:27:01.626564466
min                 0 days 00:03:03
25%          0 days 16:19:07.500000
50%                 9 days 19:45:40
75%         44 days 09:22:31.500000
max               265 days 06:34:50
Name: time2resolve, dtype: object

In [22]:
before_df["time2resolve"].describe().to_csv("public_data/time2resolve-circ-before-summary.csv", index=False)

In [23]:
before_df.to_csv("public_data/time2resolve-circ-before-all-points.csv", columns=["number", "time2resolve"], index=False)

In [24]:
after_df["time2resolve"] = after_df["resolved"] - after_df["created"]

C:\Users\tomaszkalata\AppData\Local\Temp\ipykernel_5300\861357957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_df["time2resolve"] = after_df["resolved"] - after_df["created"]


In [25]:
after_df["time2resolve"].describe()

count                         1045
mean     2 days 21:53:17.162679425
std      9 days 06:42:34.775753653
min                0 days 00:01:32
25%                0 days 02:55:04
50%                0 days 20:23:50
75%                2 days 03:20:06
max              124 days 07:48:06
Name: time2resolve, dtype: object

In [26]:
after_df["time2resolve"].describe().to_csv("public_data/time2resolve-circ-after-summary.csv", index=False)

In [27]:
after_df.to_csv("public_data/time2resolve-circ-after-all-points.csv", columns=["number", "time2resolve"], index=False)

## Chart

In [28]:
import plotly.express as px

In [38]:
before_values = before_df["time2resolve"].dt.days.tolist()

In [39]:
after_values = after_df["time2resolve"].dt.days.tolist()

In [40]:
df = pd.DataFrame(
    list(zip(before_values, after_values)),
    columns =['before', 'after'])

In [45]:
df.head()

,before,after
0,0,0
1,0,0
2,0,4
3,0,4
4,0,0


In [56]:
fig = px.box(df, x=["after", "before"], title="CAT Tickets Resolution Times")

In [57]:
fig.update_layout(xaxis_title="days", yaxis_title=None)

In [55]:
fig.write_image("charts/tests.png")